# Search Metrics

# DCG

<div>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/d7ce96a2916c5eb451c4da5a1bce54fc9a2f7894" width="200"/>
</div>

https://en.wikipedia.org/wiki/Discounted_cumulative_gain

"Discounted cumulative gain (DCG) is a measure of ranking quality. In information retrieval, it is often used to measure effectiveness of web search engine algorithms or related applications. Using a graded relevance scale of documents in a search-engine result set, DCG measures the usefulness, or gain, of a document based on its position in the result list. The gain is accumulated from the top of the result list to the bottom, with the gain of each result discounted at lower ranks."

- Sum of ((2^relevance score -1)/ log2 (position of document +1) )  for the top N documents

## How do we go from human ratings to a dcg score for each query?

- Import an example and notice the first one has disagreement, see [Raw-Data](#Raw-Data)

In [75]:
from example import *
raw_df = pd.read_csv('assets/example-dcg.csv')
raw_df[:8]

,queryid,query,document,position,rating_1,rating_2,rating_3
0,1,disagreement,doc1,1,2,2,3
1,1,disagreement,doc2,2,3,3,2
2,1,disagreement,doc3,3,0,0,2
3,1,disagreement,doc4,4,1,1,0
4,2,adhesive,doc5,1,0,0,0
5,2,adhesive,doc6,2,0,0,0
6,2,adhesive,doc7,3,0,0,0
7,2,adhesive,doc8,4,0,0,0


## Normalize relevance by taking the median rating

In [40]:
verbose_df = raw_df.copy()
verbose_df['relevance'] = raw_df[['rating_1', 'rating_2', 'rating_3']].median(axis=1).astype(int)
verbose_df[:4]

,queryid,query,document,position,rating_1,rating_2,rating_3,relevance
0,1,disagreement,doc1,1,2,2,3,2
1,1,disagreement,doc2,2,3,3,2,3
2,1,disagreement,doc3,3,0,0,2,0
3,1,disagreement,doc4,4,1,1,0,1


In [41]:
df = verbose_df[['query', 'document', 'position', 'relevance']].copy() # strip away the ratings

## How is DCG calculated? 

Sum of ((2^relevance score -1)/ log2 (position of document +1) )  for the top N documents.

<div>
<img src="https://i.imgur.com/unhes8l.png" width="200"/>
</div>

Calculate the gain by raising the relevance score to the second power (punish those bad hits!).

Don't forget the -1 😀

In [46]:
df['gain'] = np.power(2, df['relevance']) - 1
df[:4]

,query,document,position,relevance,gain
0,disagreement,doc1,1,2,3
1,disagreement,doc2,2,3,7
2,disagreement,doc3,3,0,0
3,disagreement,doc4,4,1,1


<div>
<img src="https://i.imgur.com/0kpe3Eq.png" width="200"/>
</div>

Calculate the position discount.

In [48]:
df['discount'] = np.log2(df['position']  + 1)
df[:4]

,query,document,position,relevance,gain,discount
0,disagreement,doc1,1,2,3,1.000000
1,disagreement,doc2,2,3,7,1.584963
2,disagreement,doc3,3,0,0,2.000000
3,disagreement,doc4,4,1,1,2.321928


Calculate the discounted gain: 
- (2^relevance score -1) / log2 (position of document +1)

ie gain / discount.

In [51]:
df['dg'] = np.nan_to_num(df['gain'] / df['discount']) # Replace NaN with zero
df[:4]

,query,document,position,relevance,gain,discount,dg
0,disagreement,doc1,1,2,3,1.000000,3.000000
1,disagreement,doc2,2,3,7,1.584963,4.416508
2,disagreement,doc3,3,0,0,2.000000,0.000000
3,disagreement,doc4,4,1,1,2.321928,0.430677


Sum this up cumulatively and we have dcg:

In [54]:
df['dcg'] = df.groupby(['query'])['dg'].cumsum()
df[:4]

,query,document,position,relevance,gain,discount,dg,dcg
0,disagreement,doc1,1,2,3,1.000000,3.000000,3.000000
1,disagreement,doc2,2,3,7,1.584963,4.416508,7.416508
2,disagreement,doc3,3,0,0,2.000000,0.000000,7.416508
3,disagreement,doc4,4,1,1,2.321928,0.430677,7.847185


Add in the cumulative gain and ideal rank order.

In [57]:
df['ideal_rank'] = df.groupby(['query'])['relevance'].rank(method='dense', ascending=False).astype(int)
df['cg'] = df.groupby(['query'])['relevance'].cumsum()
df[:4]

,query,document,position,relevance,gain,discount,dg,dcg,ideal_rank,cg
0,disagreement,doc1,1,2,3,1.000000,3.000000,3.000000,2,2
1,disagreement,doc2,2,3,7,1.584963,4.416508,7.416508,1,5
2,disagreement,doc3,3,0,0,2.000000,0.000000,7.416508,4,5
3,disagreement,doc4,4,1,1,2.321928,0.430677,7.847185,3,6


## What impact does relevance have on dcg?

- the lowest score

In [59]:
df[4:8]

,query,document,position,relevance,gain,discount,dg,dcg,ideal_rank,cg
4,adhesive,doc5,1,0,0,1.000000,0.0,0.0,1,0
5,adhesive,doc6,2,0,0,1.584963,0.0,0.0,1,0
6,adhesive,doc7,3,0,0,2.000000,0.0,0.0,1,0
7,adhesive,doc8,4,0,0,2.321928,0.0,0.0,1,0


- Add in a relevant document

In [60]:
df[8:12]

,query,document,position,relevance,gain,discount,dg,dcg,ideal_rank,cg
8,boots,doc9,1,0,0,1.000000,0.000000,0.000000,2,0
9,boots,doc10,2,0,0,1.584963,0.000000,0.000000,2,0
10,boots,doc11,3,0,0,2.000000,0.000000,0.000000,2,0
11,boots,doc12,4,1,1,2.321928,0.430677,0.430677,1,1


- Gradually getting better until 'frying pan' gets a perfect score of 17.93 at rank 4

In [65]:
df[4:28]

,query,document,position,relevance,gain,discount,dg,dcg,ideal_rank,cg
4,adhesive,doc5,1,0,0,1.000000,0.000000,0.000000,1,0
5,adhesive,doc6,2,0,0,1.584963,0.000000,0.000000,1,0
6,adhesive,doc7,3,0,0,2.000000,0.000000,0.000000,1,0
7,adhesive,doc8,4,0,0,2.321928,0.000000,0.000000,1,0
8,boots,doc9,1,0,0,1.000000,0.000000,0.000000,2,0
9,boots,doc10,2,0,0,1.584963,0.000000,0.000000,2,0
10,boots,doc11,3,0,0,2.000000,0.000000,0.000000,2,0
11,boots,doc12,4,1,1,2.321928,0.430677,0.430677,1,1
12,camera,doc13,1,0,0,1.000000,0.000000,0.000000,2,0
13,camera,doc14,2,0,0,1.584963,0.000000,0.000000,2,0


## Rank the queries by dcg to see low performing opportunities

- the next step would be to look in the logs for how many "adhesive" like queries there are
- what is it about "frying pan" that is enabling us to handle them so well

In [69]:
df[['query', 'dcg']].groupby("query", as_index=False).last()\
.sort_values(by='dcg', ascending=True).reset_index(drop=True)

,query,dcg
0,adhesive,0.000000
1,boots,0.430677
2,nDCG A,2.892789
3,camera,3.014736
4,door,7.000000
5,control,7.847185
6,disagreement,7.847185
7,extension cord,8.561606
8,test,9.392789
9,nDCG B,12.392789


## Compare two algorithms offline

- We can then evaluate the performance of an algorithm and compare two versions offline
- Notice that test has reordered documents 30-33 and put the most relevant document at the top

In [79]:
df[28:36]

,query,document,position,relevance,gain,discount,dg,dcg,ideal_rank,cg
28,control,doc30,1,2,3,1.000000,3.000000,3.000000,2,2
29,control,doc31,2,3,7,1.584963,4.416508,7.416508,1,5
30,control,doc32,3,0,0,2.000000,0.000000,7.416508,4,5
31,control,doc33,4,1,1,2.321928,0.430677,7.847185,3,6
32,test,doc31,1,3,7,1.000000,7.000000,7.000000,1,3
33,test,doc30,2,2,3,1.584963,1.892789,8.892789,2,5
34,test,doc33,3,1,1,2.000000,0.500000,9.392789,3,6
35,test,doc32,4,0,0,2.321928,0.000000,9.392789,4,6


# nDCG

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/b3510c9c5cf42ee8820d65335675cada51b40736)

nDCG is actual DCG/ideal DCG 

In [78]:
#TODO add examples showing why nDCG is only good as a comparison metric

## Raw Data

In [76]:
raw_df.to_json()

'{"queryid":{"0":1,"1":1,"2":1,"3":1,"4":2,"5":2,"6":2,"7":2,"8":3,"9":3,"10":3,"11":3,"12":4,"13":4,"14":4,"15":4,"16":5,"17":5,"18":5,"19":5,"20":6,"21":6,"22":6,"23":6,"24":7,"25":7,"26":7,"27":7,"28":8,"29":8,"30":8,"31":8,"32":9,"33":9,"34":9,"35":9,"36":10,"37":10,"38":10,"39":10,"40":11,"41":11,"42":11,"43":11},"query":{"0":"disagreement","1":"disagreement","2":"disagreement","3":"disagreement","4":"adhesive","5":"adhesive","6":"adhesive","7":"adhesive","8":"boots","9":"boots","10":"boots","11":"boots","12":"camera","13":"camera","14":"camera","15":"camera","16":"door","17":"door","18":"door","19":"door","20":"extension cord","21":"extension cord","22":"extension cord","23":"extension cord","24":"frying pan","25":"frying pan","26":"frying pan","27":"frying pan","28":"control","29":"control","30":"control","31":"control","32":"test","33":"test","34":"test","35":"test","36":"nDCG A","37":"nDCG A","38":"nDCG A","39":"nDCG A","40":"nDCG B","41":"nDCG B","42":"nDCG B","43":"nDCG B"},

In [77]:
raw_df

,queryid,query,document,position,rating_1,rating_2,rating_3
0,1,disagreement,doc1,1,2,2,3
1,1,disagreement,doc2,2,3,3,2
2,1,disagreement,doc3,3,0,0,2
3,1,disagreement,doc4,4,1,1,0
4,2,adhesive,doc5,1,0,0,0
5,2,adhesive,doc6,2,0,0,0
6,2,adhesive,doc7,3,0,0,0
7,2,adhesive,doc8,4,0,0,0
8,3,boots,doc9,1,0,0,0
9,3,boots,doc10,2,0,0,0
